In [1]:
# !pip install opencv-python scikit-learn pandas numpy matplotlib tensorflow-addons
# !pip install tensorflow==2.12 keras==2.12
# !pip install transformers datasets torchvision

In [1]:
import os
import kagglehub
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D, Multiply, GlobalAveragePooling2D, Dense, Input, Softmax, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import kagglehub

2025-07-02 17:46:25.262885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751478385.467787      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751478385.528181      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from transformers import ViTImageProcessor, ViTForImageClassification
import torch
from datasets import load_dataset
from PIL import Image
import requests
from sklearn.metrics import classification_report
import torch
from torch.optim import AdamW 
from transformers import ViTFeatureExtractor
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
path = kagglehub.dataset_download("adityamahimkar/iqothnccd-lung-cancer-dataset")
print("Path to dataset files:", path)
path = kagglehub.dataset_download("justinkirby/the-cancer-imaging-archive-lidcidri")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/iqothnccd-lung-cancer-dataset
Path to dataset files: /kaggle/input/the-cancer-imaging-archive-lidcidri


In [5]:
data_dir = '/kaggle/input/iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset'
test_dir = '/kaggle/input/iqothnccd-lung-cancer-dataset/Test cases'

In [6]:
img_size = (224, 224)
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = val_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 878 images belonging to 3 classes.
Found 219 images belonging to 3 classes.


In [7]:
print(train_generator.class_indices)

{'Bengin cases': 0, 'Malignant cases': 1, 'Normal cases': 2}


In [8]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

y_train = train_generator.classes  

#Compute class weightss
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

# Convert to dict for Keras
class_weights_dict = dict(enumerate(class_weights))

print("Class Weights:", class_weights_dict)

Class Weights: {0: 3.048611111111111, 1: 0.651818856718634, 2: 0.8788788788788788}


In [18]:
import torch
import numpy as np
import cv2
import os
from PIL import Image
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from transformers import DeiTForImageClassification, AutoImageProcessor

# Load DeiT processor and model
processor = AutoImageProcessor.from_pretrained('facebook/deit-base-distilled-patch16-224')

# Custom preprocessing function
def custom_preprocess(img):
    img_np = np.array(img)
    if img_np.ndim == 3 and img_np.shape[2] == 3:
        img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_np = clahe.apply(img_np)
    img_np = cv2.resize(img_np, (224, 224), interpolation=cv2.INTER_CUBIC)
    img_np = cv2.GaussianBlur(img_np, (5,5), 0)
    _, img_np = cv2.threshold(img_np, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((3,3), np.uint8)
    img_np = cv2.erode(img_np, kernel, iterations=1)
    img_np = cv2.dilate(img_np, kernel, iterations=1)
    return Image.fromarray(img_np)

# Transforms
train_transform = transforms.Compose([
    transforms.Lambda(custom_preprocess),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

val_transform = transforms.Compose([
    transforms.Lambda(custom_preprocess),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

# Data paths
data_dir = '/kaggle/input/iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset'
test_dir = '/kaggle/input/iqothnccd-lung-cancer-dataset/Test cases'

# Load dataset and split
full_dataset = datasets.ImageFolder(root=data_dir)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Apply transforms
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load model with correct number of classes
num_classes = len(full_dataset.classes)
model = DeiTForImageClassification.from_pretrained(
    'facebook/deit-base-distilled-patch16-224',
    num_labels=num_classes
)

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f"Dataset loaded: {len(full_dataset)} samples")
print(f"Classes: {full_dataset.classes}")
print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")
print(f"Device: {device}")

Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Dataset loaded: 1097 samples
Classes: ['Bengin cases', 'Malignant cases', 'Normal cases']
Train: 877, Val: 220
Device: cuda


In [19]:
model = DeiTForImageClassification.from_pretrained(
    'facebook/deit-base-distilled-patch16-224',
    num_labels=3
)

Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
import copy

best_val_loss = float('inf')
best_model_state = None

In [21]:
from torch.utils.data import random_split

full_dataset = datasets.ImageFolder(root=data_dir)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

In [22]:
import torch
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


train_labels = [full_dataset.samples[i][1] for i in train_dataset.indices]


class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

optimizer = AdamW(model.parameters(), lr=5e-5)

best_val_loss = float('inf')
best_model_state = None

In [23]:
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
import numpy as np
import torch
from torch.utils.data import Subset
import copy
from tqdm import tqdm

# Training configuration
CONFIG = {
    'n_splits': 5,
    'epochs': 15,
    'batch_size': 32,
    'lr': 2e-5,
    'weight_decay': 0.01,
    'patience': 3,
    'min_delta': 1e-4
}

def train_epoch(model, loader, optimizer, loss_fn, scheduler, device):
    model.train()
    total_loss, correct, total = 0, 0, 0
    
    for inputs, labels in tqdm(loader, desc="Training", leave=False):
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        
        optimizer.zero_grad()
        outputs = model(inputs).logits
        loss = loss_fn(outputs, labels)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    return total_loss / len(loader), correct / total

def validate(model, loader, loss_fn, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="Validation", leave=False):
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            
            outputs = model(inputs).logits
            loss = loss_fn(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    return total_loss / len(loader), correct / total

# Prepare stratified splits
all_labels = [sample[1] for sample in full_dataset.samples]
skf = StratifiedKFold(n_splits=CONFIG['n_splits'], shuffle=True, random_state=42)
fold_results = []

print(f"Starting {CONFIG['n_splits']}-fold cross-validation")
print(f"Total samples: {len(full_dataset)}")

for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(all_labels)), all_labels), 1):
    print(f"\n{'='*50}")
    print(f"FOLD {fold}/{CONFIG['n_splits']}")
    print(f"{'='*50}")
    
    # Create subsets
    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)
    
    # Apply transforms
    train_subset.dataset.transform = train_transform
    val_subset.dataset.transform = val_transform
    
    # Data loaders with optimizations
    train_loader = DataLoader(
        train_subset, 
        batch_size=CONFIG['batch_size'], 
        shuffle=True, 
        num_workers=2, 
        pin_memory=True
    )
    val_loader = DataLoader(
        val_subset, 
        batch_size=CONFIG['batch_size'], 
        shuffle=False, 
        num_workers=2, 
        pin_memory=True
    )
    
    # Compute balanced class weights
    train_labels_fold = [all_labels[i] for i in train_idx]
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(train_labels_fold),
        y=train_labels_fold
    )
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
    loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=0.1)
    
    # Initialize model
    model = DeiTForImageClassification.from_pretrained(
        'facebook/deit-base-distilled-patch16-224',
        num_labels=num_classes
    ).to(device)
    
    # Optimizer and scheduler
    optimizer = AdamW(
        model.parameters(), 
        lr=CONFIG['lr'], 
        weight_decay=CONFIG['weight_decay']
    )
    total_steps = len(train_loader) * CONFIG['epochs']
    scheduler = CosineAnnealingLR(optimizer, T_max=total_steps)
    
    # Training loop with early stopping
    best_val_acc = 0
    best_model_state = None
    patience_counter = 0
    
    print(f"Train samples: {len(train_subset)}, Val samples: {len(val_subset)}")
    
    fold_history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    for epoch in range(CONFIG['epochs']):
        print(f"\nEpoch {epoch+1}/{CONFIG['epochs']}")
        
        # Train
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, loss_fn, scheduler, device)
        
        # Validate
        val_loss, val_acc = validate(model, val_loader, loss_fn, device)
        
        # Store history
        fold_history['train_loss'].append(train_loss)
        fold_history['train_acc'].append(train_acc)
        fold_history['val_loss'].append(val_loss)
        fold_history['val_acc'].append(val_acc)
        
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        # Early stopping check
        if val_acc > best_val_acc + CONFIG['min_delta']:
            best_val_acc = val_acc
            best_model_state = copy.deepcopy(model.state_dict())
            patience_counter = 0
            print("✅ New best model!")
        else:
            patience_counter += 1
            if patience_counter >= CONFIG['patience']:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break
    
    # Save best model for this fold
    torch.save(best_model_state, f"best_model_fold{fold}.pth")
    print(f"Best accuracy for fold {fold}: {best_val_acc:.4f}")
    
    fold_results.append({
        'fold': fold,
        'best_accuracy': best_val_acc,
        'history': fold_history
    })

# Final results
accuracies = [result['best_accuracy'] for result in fold_results]
print(f"\n{'='*60}")
print("CROSS-VALIDATION RESULTS")
print(f"{'='*60}")
for i, acc in enumerate(accuracies, 1):
    print(f"Fold {i}: {acc:.4f}")
print(f"{'='*60}")
print(f"Mean Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
print(f"Best Fold: {np.argmax(accuracies) + 1} ({np.max(accuracies):.4f})")
print(f"{'='*60}")

# Clean up GPU memory
torch.cuda.empty_cache()

Starting 5-fold cross-validation
Total samples: 1097

FOLD 1/5


Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train samples: 877, Val samples: 220

Epoch 1/15


Train Loss: 0.9560, Train Acc: 0.6089
Val Loss: 0.8098, Val Acc: 0.7955
✅ New best model!

Epoch 2/15


Train Loss: 0.5586, Train Acc: 0.9168
Val Loss: 0.6138, Val Acc: 0.9591
✅ New best model!

Epoch 3/15


Train Loss: 0.4531, Train Acc: 0.9875
Val Loss: 0.5849, Val Acc: 0.9636
✅ New best model!

Epoch 4/15


Train Loss: 0.4260, Train Acc: 0.9989
Val Loss: 0.5652, Val Acc: 0.9591

Epoch 5/15


Train Loss: 0.4248, Train Acc: 1.0000
Val Loss: 0.5527, Val Acc: 0.9636

Epoch 6/15


Train Loss: 0.4144, Train Acc: 1.0000
Val Loss: 0.5508, Val Acc: 0.9727
✅ New best model!

Epoch 7/15


Train Loss: 0.4116, Train Acc: 1.0000
Val Loss: 0.5507, Val Acc: 0.9727

Epoch 8/15


Train Loss: 0.4073, Train Acc: 1.0000
Val Loss: 0.5510, Val Acc: 0.9727

Epoch 9/15


Train Loss: 0.4096, Train Acc: 1.0000
Val Loss: 0.5502, Val Acc: 0.9773
✅ New best model!

Epoch 10/15


Train Loss: 0.4102, Train Acc: 1.0000
Val Loss: 0.5500, Val Acc: 0.9773

Epoch 11/15


Train Loss: 0.4135, Train Acc: 1.0000
Val Loss: 0.5501, Val Acc: 0.9773

Epoch 12/15


Train Loss: 0.4091, Train Acc: 1.0000
Val Loss: 0.5496, Val Acc: 0.9773
Early stopping triggered after 12 epochs
Best accuracy for fold 1: 0.9773

FOLD 2/5


Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train samples: 877, Val samples: 220

Epoch 1/15


Train Loss: 0.9548, Train Acc: 0.5941
Val Loss: 0.8020, Val Acc: 0.8500
✅ New best model!

Epoch 2/15


Train Loss: 0.5680, Train Acc: 0.9065
Val Loss: 0.6495, Val Acc: 0.8909
✅ New best model!

Epoch 3/15


Train Loss: 0.4612, Train Acc: 0.9886
Val Loss: 0.6444, Val Acc: 0.9364
✅ New best model!

Epoch 4/15


Train Loss: 0.4311, Train Acc: 0.9989
Val Loss: 0.5841, Val Acc: 0.9682
✅ New best model!

Epoch 5/15


Train Loss: 0.4136, Train Acc: 1.0000
Val Loss: 0.5734, Val Acc: 0.9682

Epoch 6/15


Train Loss: 0.4133, Train Acc: 1.0000
Val Loss: 0.5650, Val Acc: 0.9682

Epoch 7/15


Train Loss: 0.4114, Train Acc: 1.0000
Val Loss: 0.5642, Val Acc: 0.9682
Early stopping triggered after 7 epochs
Best accuracy for fold 2: 0.9682

FOLD 3/5


Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train samples: 878, Val samples: 219

Epoch 1/15


Train Loss: 0.9247, Train Acc: 0.6503
Val Loss: 0.8497, Val Acc: 0.7534
✅ New best model!

Epoch 2/15


Train Loss: 0.5551, Train Acc: 0.9214
Val Loss: 0.6353, Val Acc: 0.9635
✅ New best model!

Epoch 3/15


Train Loss: 0.4483, Train Acc: 0.9954
Val Loss: 0.6342, Val Acc: 0.9269

Epoch 4/15


Train Loss: 0.4291, Train Acc: 0.9966
Val Loss: 0.5842, Val Acc: 0.9635

Epoch 5/15


Train Loss: 0.4206, Train Acc: 0.9989
Val Loss: 0.5643, Val Acc: 0.9817
✅ New best model!

Epoch 6/15


Train Loss: 0.4190, Train Acc: 1.0000
Val Loss: 0.5663, Val Acc: 0.9772

Epoch 7/15


Train Loss: 0.4158, Train Acc: 1.0000
Val Loss: 0.5557, Val Acc: 0.9817

Epoch 8/15


Train Loss: 0.4137, Train Acc: 1.0000
Val Loss: 0.5603, Val Acc: 0.9772
Early stopping triggered after 8 epochs
Best accuracy for fold 3: 0.9817

FOLD 4/5


Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train samples: 878, Val samples: 219

Epoch 1/15


Train Loss: 0.9135, Train Acc: 0.6492
Val Loss: 0.7835, Val Acc: 0.8128
✅ New best model!

Epoch 2/15


Train Loss: 0.5737, Train Acc: 0.9123
Val Loss: 0.6597, Val Acc: 0.8995
✅ New best model!

Epoch 3/15


Train Loss: 0.4515, Train Acc: 0.9943
Val Loss: 0.6044, Val Acc: 0.9315
✅ New best model!

Epoch 4/15


Train Loss: 0.4227, Train Acc: 0.9989
Val Loss: 0.5690, Val Acc: 0.9635
✅ New best model!

Epoch 5/15


Train Loss: 0.4234, Train Acc: 1.0000
Val Loss: 0.5622, Val Acc: 0.9680
✅ New best model!

Epoch 6/15


Train Loss: 0.4109, Train Acc: 1.0000
Val Loss: 0.5603, Val Acc: 0.9635

Epoch 7/15


Train Loss: 0.4113, Train Acc: 1.0000
Val Loss: 0.5632, Val Acc: 0.9635

Epoch 8/15


Train Loss: 0.4096, Train Acc: 1.0000
Val Loss: 0.5615, Val Acc: 0.9635
Early stopping triggered after 8 epochs
Best accuracy for fold 4: 0.9680

FOLD 5/5


Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train samples: 878, Val samples: 219

Epoch 1/15


Train Loss: 0.9145, Train Acc: 0.6401
Val Loss: 0.6962, Val Acc: 0.9132
✅ New best model!

Epoch 2/15


Train Loss: 0.5371, Train Acc: 0.9431
Val Loss: 0.5862, Val Acc: 0.9772
✅ New best model!

Epoch 3/15


Train Loss: 0.4436, Train Acc: 0.9943
Val Loss: 0.5497, Val Acc: 0.9726

Epoch 4/15


Train Loss: 0.4214, Train Acc: 0.9989
Val Loss: 0.5429, Val Acc: 0.9817
✅ New best model!

Epoch 5/15


Train Loss: 0.4189, Train Acc: 1.0000
Val Loss: 0.5379, Val Acc: 0.9863
✅ New best model!

Epoch 6/15


Train Loss: 0.4144, Train Acc: 1.0000
Val Loss: 0.5266, Val Acc: 1.0000
✅ New best model!

Epoch 7/15


Train Loss: 0.4174, Train Acc: 1.0000
Val Loss: 0.5293, Val Acc: 0.9909

Epoch 8/15


Train Loss: 0.4130, Train Acc: 1.0000
Val Loss: 0.5295, Val Acc: 0.9909

Epoch 9/15


Train Loss: 0.4148, Train Acc: 1.0000
Val Loss: 0.5297, Val Acc: 0.9909
Early stopping triggered after 9 epochs
Best accuracy for fold 5: 1.0000

CROSS-VALIDATION RESULTS
Fold 1: 0.9773
Fold 2: 0.9682
Fold 3: 0.9817
Fold 4: 0.9680
Fold 5: 1.0000
Mean Accuracy: 0.9790 ± 0.0117
Best Fold: 5 (1.0000)


EVALUATION RESULTS
                 precision    recall  f1-score   support

   Bengin cases       0.92      1.00      0.96        24
Malignant cases       1.00      1.00      1.00       112
   Normal cases       1.00      0.98      0.99        83

       accuracy                           0.99       219
      macro avg       0.97      0.99      0.98       219
   weighted avg       0.99      0.99      0.99       219

Overall Accuracy: 1.0000


Bengin cases Accuracy: 1.0000
Malignant cases Accuracy: 1.0000
Normal cases Accuracy: 0.9759


In [25]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Evaluation
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        logits = model(inputs).logits
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print detailed metrics
print("="*60)
print("EVALUATION RESULTS")
print("="*60)
print(classification_report(all_labels, all_preds, target_names=full_dataset.classes))

accuracy = accuracy_score(all_labels, all_labels)
print(f"Overall Accuracy: {accuracy:.4f}")

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=full_dataset.classes, 
            yticklabels=full_dataset.classes)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Per-class accuracy
for i, class_name in enumerate(full_dataset.classes):
    class_mask = np.array(all_labels) == i
    if class_mask.sum() > 0:
        class_acc = (np.array(all_preds)[class_mask] == i).mean()
        print(f"{class_name} Accuracy: {class_acc:.4f}")

In [26]:
torch.save(best_model_state, 'deit_best_model.pt')